In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/01 01:36:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/01 01:36:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [80]:
spark.version

'3.3.2'

In [11]:
from pyspark.sql import types

In [25]:
import pyarrow.parquet as pq

In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-01 01:39:09--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.209.72, 54.230.209.200, 54.230.209.140, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.209.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  73.1MB/s    in 0.8s    

2025-03-01 01:39:10 (73.1 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [32]:
df_yellow_2024.head()

25/03/01 02:19:16 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 2, schema size: 18
CSV file: file:///home/suman/batch-pyspark/data/yellow_tripdata_2024-10.parquet


Row(VendorID=None, tpep_pickup_datetime=None, tpep_dropoff_datetime=None, passenger_count=None, trip_distance=None, RatecodeID=None, store_and_fwd_flag=None, PULocationID=None, DOLocationID=None, payment_type=None, fare_amount=None, extra=None, mta_tax=None, tip_amount=None, tolls_amount=None, improvement_surcharge=None, total_amount=None, congestion_surcharge=None)

In [20]:
import pandas as pd

In [26]:
df_yellow_2024pd = pd.read_parquet('yellow_tripdata_2024-10.parquet')

In [27]:
df_yellow_2024pd.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-10-01 00:30:44,2024-10-01 00:48:26,1.0,3.0,1.0,N,162,246,1,18.4,1.0,0.5,1.5,0.0,1.0,24.9,2.5,0.0
1,1,2024-10-01 00:12:20,2024-10-01 00:25:25,1.0,2.2,1.0,N,48,236,1,14.2,3.5,0.5,3.8,0.0,1.0,23.0,2.5,0.0
2,1,2024-10-01 00:04:46,2024-10-01 00:13:52,1.0,2.7,1.0,N,142,24,1,13.5,3.5,0.5,3.7,0.0,1.0,22.2,2.5,0.0
3,1,2024-10-01 00:12:10,2024-10-01 00:23:01,1.0,3.1,1.0,N,233,75,1,14.2,3.5,0.5,2.0,0.0,1.0,21.2,2.5,0.0
4,1,2024-10-01 00:30:22,2024-10-01 00:30:39,1.0,0.0,1.0,N,262,262,3,3.0,3.5,0.5,0.0,0.0,1.0,8.0,2.5,0.0


In [65]:
DY_1024 = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [70]:
DY_1024.repartition(4).write  \
    .mode("overwrite").parquet("./yellow2024/HW-output-pq/")

In [71]:
DY_1024.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [44]:
from pyspark.sql import functions as F

In [72]:
DY_1024.registerTempTable('yellow1024')

In [73]:
spark.sql("""
SELECT
    count(*)as total_trips
FROM
    yellow1024
WHERE 
    to_date(tpep_pickup_datetime)= '2024-10-15';
""").show()

+-----------+
|total_trips|
+-----------+
|     128893|
+-----------+



In [79]:
spark.sql("""
SELECT
    COUNT(*) AS total_trips
FROM
    yellow1024
WHERE 
    EXTRACT(YEAR FROM tpep_pickup_datetime) = 2024 AND
    EXTRACT(MONTH FROM tpep_pickup_datetime) = 10 AND
    EXTRACT(DAY FROM tpep_pickup_datetime) = 15
""").show()

+-----------+
|total_trips|
+-----------+
|     128893|
+-----------+



In [74]:
DY_1024 \
    .withColumn('pickup_date', F.to_date(DY_1024.tpep_pickup_datetime)) \
    .filter("pickup_date = '2024-10-15'") \
    .count()

128893

In [75]:
DY_1024 \
    .withColumn('duration', DY_1024.tpep_dropoff_datetime.cast('long') - DY_1024.tpep_pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(DY_1024.tpep_pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2024-10-16|       585424|
| 2024-10-03|       515970|
| 2024-10-22|       495938|
| 2024-10-18|       413405|
| 2024-10-21|       323634|
+-----------+-------------+



In [78]:
spark.sql("""
    WITH duration_data AS (
        SELECT *, 
               CAST(tpep_dropoff_datetime AS LONG) - CAST(tpep_pickup_datetime AS LONG) AS duration, 
               TO_DATE(tpep_pickup_datetime) AS pickup_date
          FROM yellow1024
    ) 
    SELECT pickup_date, 
           ROUND(MAX(duration)/3600,2) AS max_duration
      FROM duration_data
    GROUP BY pickup_date
    ORDER BY max_duration DESC
    LIMIT 5;
""").show()


+-----------+------------+
|pickup_date|max_duration|
+-----------+------------+
| 2024-10-16|      162.62|
| 2024-10-03|      143.33|
| 2024-10-22|      137.76|
| 2024-10-18|      114.83|
| 2024-10-21|        89.9|
+-----------+------------+



In [54]:
df_zones = spark.read.parquet('/home/suman/batch-pyspark/zones')

In [55]:
df_zones.columns

['LocationID', 'Borough', 'Zone', 'service_zone']

In [57]:
df_zones.registerTempTable('zones')

In [76]:
spark.sql("""
    SELECT
        CONCAT(zp.Zone, ' / ', zd.Zone) AS PickUp_DropOff_Location,
        COUNT(1) AS trip_count
    FROM 
        yellow1024 y
        LEFT JOIN zones zp ON y.PULocationID = zp.LocationID
        LEFT JOIN zones zd ON y.DOLocationID = zd.LocationID
    GROUP BY 
        PickUp_DropOff_Location
    ORDER BY 
        trip_count
    LIMIT 5;
""").show()

+-----------------------+----------+
|PickUp_DropOff_Location|trip_count|
+-----------------------+----------+
|   Borough Park / Pe...|         1|
|   Melrose South / H...|         1|
|   Bay Ridge / Green...|         1|
|   East Tremont / St...|         1|
|   East Harlem South...|         1|
+-----------------------+----------+



In [77]:
# Join df_yellow_2024 with zones for pickup and drop-off locations
df_joined = DY_1024 \
    .join(df_zones.alias("zp"), DY_1024.PULocationID == F.col("zp.LocationID"), "left") \
    .join(df_zones.alias("zd"), DY_1024.DOLocationID == F.col("zd.LocationID"), "left")
          
# Concatenate Zone names for pickup and drop-off locations
df_with_location = df_joined.withColumn(
    "PickUp_DropOff_Location",
    F.concat(F.col("zp.Zone"), F.lit(" / "), F.col("zd.Zone"))
)          
# Group by the concatenated pickup and drop-off location, and count trips
df_grouped = df_with_location.groupBy("PickUp_DropOff_Location").agg(
    F.count("*").alias("trip_count")
)   
 # Order by trip_count in descending order and limit to top 5
df_least5 = df_grouped.orderBy(F.col("trip_count").asc()).limit(5)

# Show the results
df_least5.show()

+-----------------------+----------+
|PickUp_DropOff_Location|trip_count|
+-----------------------+----------+
|   Borough Park / Pe...|         1|
|   Melrose South / H...|         1|
|   Bay Ridge / Green...|         1|
|   East Tremont / St...|         1|
|   East Harlem South...|         1|
+-----------------------+----------+

